In [3]:
# function ClickConnect(){
#     console.log("Clicked on connect button"); 
#     document.querySelector("#ok").click()
# }
# setInterval(ClickConnect,70006)


import pickle
import os
import tarfile
import urllib
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDRegressor, LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.base import clone


        

In [4]:
raw_dat = pd.read_pickle("appml-assignment1-dataset.pkl")
raw_dat

{'X':                      date  CAD-open  CAD-high  ...  ZCN-high  ZCN-low  ZCN-close
 18412 2015-01-14 17:15:00  1.195500  1.196065  ...     19.05    18.95      19.05
 14220 2014-05-12 19:00:00  1.089745  1.089975  ...     19.81    19.76      19.81
 36252 2017-11-29 19:00:00  1.286515  1.287155  ...     21.60    21.58      21.58
 13653 2014-04-09 03:00:00  1.092575  1.093375  ...     19.40    19.38      19.40
 49163 2020-01-02 12:00:00  1.299845  1.300430  ...     23.01    23.00      23.01
 ...                   ...       ...       ...  ...       ...      ...        ...
 13544 2014-04-02 14:00:00  1.102880  1.103120  ...     19.47    19.45      19.47
 24384 2015-12-31 13:00:00  1.382800  1.384600  ...     17.59    17.59      17.59
 32148 2017-04-04 17:15:00  1.340350  1.340570  ...     21.11    21.09      21.10
 27276 2016-06-21 02:00:00  1.278885  1.279525  ...     19.05    19.01      19.01
 20993 2015-06-16 03:00:00  1.231620  1.233640  ...     20.03    20.00      20.03
 
 [39640 r

In [5]:
raw_dat['X']

,date,CAD-open,CAD-high,CAD-low,CAD-close,CHF-open,CHF-high,CHF-low,CHF-close,CNH-open,CNH-high,CNH-low,CNH-close,CZK-open,CZK-high,CZK-low,CZK-close,DKK-open,DKK-high,DKK-low,DKK-close,DVS-open,DVS-high,DVS-low,DVS-close,EIS-open,EIS-high,EIS-low,EIS-close,ERUS-open,ERUS-high,ERUS-low,ERUS-close,EUR-open,EUR-high,EUR-low,EUR-close,EWA-open,EWA-high,EWA-low,...,SEK-open,SEK-high,SEK-low,SEK-close,SGD-open,SGD-high,SGD-low,SGD-close,SPX-open,SPX-high,SPX-low,SPX-close,SPY-open,SPY-high,SPY-low,SPY-close,TYVIX-open,TYVIX-high,TYVIX-low,TYVIX-close,USO-open,USO-high,USO-low,USO-close,VCN-open,VCN-high,VCN-low,VCN-close,VGK-open,VGK-high,VGK-low,VGK-close,VIX-open,VIX-high,VIX-low,VIX-close,ZCN-open,ZCN-high,ZCN-low,ZCN-close
18412,2015-01-14 17:15:00,1.195500,1.196065,1.195430,1.195735,1.018795,1.018870,1.018360,1.018665,6.195550,6.200090,6.194520,6.195300,23.79155,23.79325,23.78650,23.79075,6.310170,6.310400,6.307275,6.310115,24.5,24.5,24.5,24.5,46.92,46.94,46.74,46.82,23.24,23.42,23.24,23.38,1.178750,1.179310,1.178700,1.178945,21.88,21.99,21.87,...,8.084575,8.087140,8.078765,8.086895,1.334720,1.335385,1.333525,1.333795,2000.51,2012.66,2000.38,2011.03,199.78,201.05,199.75,200.90,NaN,NaN,NaN,NaN,18.17,18.33,18.15,18.27,28.11,28.19,28.11,28.19,50.95,51.24,50.94,51.15,21.88,21.88,21.47,21.48,18.95,19.05,18.95,19.05
14220,2014-05-12 19:00:00,1.089745,1.089975,1.089720,1.089935,0.887755,0.887810,0.887695,0.887785,6.243690,6.244350,6.241950,6.242055,19.92555,19.92700,19.92350,19.92605,5.426080,5.426495,5.425605,5.426335,49.1,49.1,49.1,49.1,51.82,51.90,51.82,51.85,37.06,37.12,37.04,37.08,1.375620,1.375745,1.375520,1.375565,26.58,26.62,26.56,...,6.568195,6.569570,6.568020,6.569270,1.250335,1.250915,1.250285,1.250740,1896.02,1897.13,1895.13,1896.74,189.73,189.88,189.63,189.78,NaN,NaN,NaN,NaN,36.61,36.64,36.60,36.61,29.42,29.47,29.42,29.47,60.69,60.79,60.68,60.77,12.19,12.23,12.15,12.23,19.78,19.81,19.76,19.81
36252,2017-11-29 19:00:00,1.286515,1.287155,1.286305,1.286595,0.984525,0.984865,0.984180,0.984205,6.615300,6.616985,6.613690,6.614625,21.50100,21.50970,21.49385,21.49615,6.280025,6.282290,6.277740,6.278180,94.6,94.6,94.6,94.6,49.30,49.32,49.27,49.27,34.23,34.25,34.12,34.13,1.185220,1.185600,1.184775,1.185450,22.83,22.84,22.81,...,8.368720,8.374025,8.366005,8.368740,1.347235,1.347815,1.347075,1.347175,2624.17,2627.35,2622.05,2625.90,262.59,262.88,262.36,262.73,3.63,3.64,3.62,3.62,11.48,11.50,11.47,11.48,32.39,32.39,32.36,32.38,58.50,58.51,58.43,58.44,10.66,10.70,10.65,10.70,21.60,21.60,21.58,21.58
13653,2014-04-09 03:00:00,1.092575,1.093375,1.092380,1.092760,0.884270,0.884315,0.882895,0.883050,6.198035,6.200785,6.197025,6.198805,19.88400,19.88660,19.85450,19.85655,5.414685,5.415145,5.407780,5.408485,18.8,18.8,18.8,18.8,53.09,53.09,53.09,53.09,35.86,35.90,35.76,35.82,1.378820,1.380570,1.378725,1.380415,26.34,26.38,26.30,...,6.496630,6.508525,6.467240,6.504800,1.250775,1.250775,1.249260,1.249300,1850.30,1853.76,1848.52,1852.14,184.90,185.28,184.69,185.11,NaN,NaN,NaN,NaN,36.95,36.98,36.90,36.92,28.89,28.92,28.89,28.91,58.96,59.03,58.91,58.97,14.86,14.90,14.83,14.89,19.38,19.40,19.38,19.40
49163,2020-01-02 12:00:00,1.299845,1.300430,1.299275,1.299670,0.972185,0.973320,0.972180,0.972830,6.963825,6.964945,6.962120,6.963350,22.69175,22.69605,22.66300,22.67380,6.691980,6.693925,6.688980,6.690245,16.0,16.0,16.0,16.0,57.87,57.87,57.87,57.87,43.60,43.81,43.60,43.81,1.116620,1.117195,1.116360,1.116995,22.68,22.69,22.67,...,9.369135,9.373140,9.366095,9.366190,1.347655,1.347835,1.347525,1.347570,3240.40,3244.13,3239.45,3243.37,323.06,323.46,322.92,323.36,4.11,4.11,4.10,4.11,12.74,12.76,12.72,12.75,34.42,34.43,34.41,34.41,58.87,58.89,58.84,58.85,12.93,13.00,12.69,12.73,23.01,23.01,23.00,23.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [6]:
raw_dat['X'].keys()

Index(['date', 'CAD-open', 'CAD-high', 'CAD-low', 'CAD-close', 'CHF-open',
       'CHF-high', 'CHF-low', 'CHF-close', 'CNH-open',
       ...
       'VGK-low', 'VGK-close', 'VIX-open', 'VIX-high', 'VIX-low', 'VIX-close',
       'ZCN-open', 'ZCN-high', 'ZCN-low', 'ZCN-close'],
      dtype='object', length=189)

In [7]:
# There is a few col with too much nan, which doesnt really contribute so I think I will drop it
raw_dat['X'] = raw_dat['X'].drop("date", axis=1)
# raw_dat['X'] = raw_dat['X'].drop("TYVIX-open", axis=1)
# raw_dat['X'] = raw_dat['X'].drop("TYVIX-high", axis=1)
# raw_dat['X'] = raw_dat['X'].drop("TYVIX-low", axis=1)
# raw_dat['X'] = raw_dat['X'].drop("TYVIX-close", axis=1)

In [8]:
raw_dat['y']

18413    1.195960
14221    1.090230
36253    1.286935
13654    1.093935
49164    1.299955
           ...   
13545    1.103615
24385    1.384595
32149    1.340235
27277    1.278685
20994    1.234685
Name: CAD-high, Length: 39640, dtype: float64

In [9]:
X = raw_dat['X']
y = raw_dat['y']

In [10]:
X[X.isnull().any(axis=1)]

,CAD-open,CAD-high,CAD-low,CAD-close,CHF-open,CHF-high,CHF-low,CHF-close,CNH-open,CNH-high,CNH-low,CNH-close,CZK-open,CZK-high,CZK-low,CZK-close,DKK-open,DKK-high,DKK-low,DKK-close,DVS-open,DVS-high,DVS-low,DVS-close,EIS-open,EIS-high,EIS-low,EIS-close,ERUS-open,ERUS-high,ERUS-low,ERUS-close,EUR-open,EUR-high,EUR-low,EUR-close,EWA-open,EWA-high,EWA-low,EWA-close,...,SEK-open,SEK-high,SEK-low,SEK-close,SGD-open,SGD-high,SGD-low,SGD-close,SPX-open,SPX-high,SPX-low,SPX-close,SPY-open,SPY-high,SPY-low,SPY-close,TYVIX-open,TYVIX-high,TYVIX-low,TYVIX-close,USO-open,USO-high,USO-low,USO-close,VCN-open,VCN-high,VCN-low,VCN-close,VGK-open,VGK-high,VGK-low,VGK-close,VIX-open,VIX-high,VIX-low,VIX-close,ZCN-open,ZCN-high,ZCN-low,ZCN-close
18412,1.195500,1.196065,1.195430,1.195735,1.018795,1.018870,1.018360,1.018665,6.195550,6.200090,6.194520,6.195300,23.79155,23.79325,23.78650,23.79075,6.310170,6.310400,6.307275,6.310115,24.5,24.5,24.5,24.5,46.92,46.94,46.74,46.82,23.24,23.42,23.24,23.38,1.178750,1.179310,1.178700,1.178945,21.88,21.99,21.87,21.95,...,8.084575,8.087140,8.078765,8.086895,1.334720,1.335385,1.333525,1.333795,2000.51,2012.66,2000.38,2011.03,199.78,201.05,199.75,200.90,NaN,NaN,NaN,NaN,18.17,18.33,18.15,18.27,28.11,28.19,28.11,28.19,50.95,51.24,50.94,51.15,21.88,21.88,21.47,21.48,18.95,19.05,18.95,19.05
14220,1.089745,1.089975,1.089720,1.089935,0.887755,0.887810,0.887695,0.887785,6.243690,6.244350,6.241950,6.242055,19.92555,19.92700,19.92350,19.92605,5.426080,5.426495,5.425605,5.426335,49.1,49.1,49.1,49.1,51.82,51.90,51.82,51.85,37.06,37.12,37.04,37.08,1.375620,1.375745,1.375520,1.375565,26.58,26.62,26.56,26.62,...,6.568195,6.569570,6.568020,6.569270,1.250335,1.250915,1.250285,1.250740,1896.02,1897.13,1895.13,1896.74,189.73,189.88,189.63,189.78,NaN,NaN,NaN,NaN,36.61,36.64,36.60,36.61,29.42,29.47,29.42,29.47,60.69,60.79,60.68,60.77,12.19,12.23,12.15,12.23,19.78,19.81,19.76,19.81
13653,1.092575,1.093375,1.092380,1.092760,0.884270,0.884315,0.882895,0.883050,6.198035,6.200785,6.197025,6.198805,19.88400,19.88660,19.85450,19.85655,5.414685,5.415145,5.407780,5.408485,18.8,18.8,18.8,18.8,53.09,53.09,53.09,53.09,35.86,35.90,35.76,35.82,1.378820,1.380570,1.378725,1.380415,26.34,26.38,26.30,26.34,...,6.496630,6.508525,6.467240,6.504800,1.250775,1.250775,1.249260,1.249300,1850.30,1853.76,1848.52,1852.14,184.90,185.28,184.69,185.11,NaN,NaN,NaN,NaN,36.95,36.98,36.90,36.92,28.89,28.92,28.89,28.91,58.96,59.03,58.91,58.97,14.86,14.90,14.83,14.89,19.38,19.40,19.38,19.40
11417,1.054465,1.055010,1.052845,1.052960,0.906270,0.906915,0.906265,0.906780,6.081260,6.081515,6.080505,6.081105,20.14550,20.15750,20.14550,20.15050,5.494800,5.497580,5.494795,5.496905,66.6,66.6,66.6,66.6,47.81,47.82,47.78,47.79,43.02,43.06,42.94,42.98,1.357405,1.357405,1.356725,1.356910,25.64,25.65,25.58,25.59,...,6.573875,6.578560,6.571500,6.577235,1.251580,1.252345,1.251460,1.251640,1807.44,1808.42,1802.97,1803.02,181.08,181.22,180.63,180.69,NaN,NaN,NaN,NaN,33.67,33.70,33.66,33.69,26.72,26.72,26.72,26.72,56.96,56.97,56.89,56.90,12.51,12.90,12.49,12.90,18.10,18.13,18.09,18.09
1380,0.998500,0.998500,0.997175,0.997675,0.914125,0.914300,0.912525,0.914300,NaN,NaN,NaN,NaN,18.86575,18.87640,18.81405,18.87130,5.658825,5.660550,5.647675,5.660550,19.2,19.2,19.2,19.2,42.59,42.59,42.42,42.42,47.78,47.84,47.60,47.62,1.314725,1.317275,1.314325,1.314325,23.29,23.30,23.24,23.24,...,6.762650,6.763000,6.747500,6.763000,1.251700,1.251900,1.250650,1.251750,1369.70,1369.71,1369.56,1369.57,137.31,137.36,136.98,136.99,NaN,NaN,NaN,NaN,39.14,39.21,39.13,39.18,NaN,NaN,NaN,NaN,43.78,43.83,43.75,43.78,19.67,19.74,19.53,19.55,16.27,16.29,16.27,16.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6564,1.006975,1.008225,1.005475,1.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train2    

,CAD-open,CAD-high,CAD-low,CAD-close,CHF-open,CHF-high,CHF-low,CHF-close,CNH-open,CNH-high,CNH-low,CNH-close,CZK-open,CZK-high,CZK-low,CZK-close,DKK-open,DKK-high,DKK-low,DKK-close,DVS-open,DVS-high,DVS-low,DVS-close,EIS-open,EIS-high,EIS-low,EIS-close,ERUS-open,ERUS-high,ERUS-low,ERUS-close,EUR-open,EUR-high,EUR-low,EUR-close,EWA-open,EWA-high,EWA-low,EWA-close,...,SEK-open,SEK-high,SEK-low,SEK-close,SGD-open,SGD-high,SGD-low,SGD-close,SPX-open,SPX-high,SPX-low,SPX-close,SPY-open,SPY-high,SPY-low,SPY-close,TYVIX-open,TYVIX-high,TYVIX-low,TYVIX-close,USO-open,USO-high,USO-low,USO-close,VCN-open,VCN-high,VCN-low,VCN-close,VGK-open,VGK-high,VGK-low,VGK-close,VIX-open,VIX-high,VIX-low,VIX-close,ZCN-open,ZCN-high,ZCN-low,ZCN-close
47899,1.313380,1.313870,1.313240,1.313740,0.986880,0.988170,0.986880,0.987955,7.080455,7.083135,7.080395,7.080540,23.07925,23.09550,23.07890,23.09050,6.714085,6.717750,6.713600,6.716410,34.1,34.1,34.1,34.1,54.80,54.80,54.80,54.80,39.91,40.00,39.91,40.00,1.112760,1.112895,1.112165,1.112385,22.35,22.36,22.32,22.35,...,9.712690,9.719810,9.710825,9.716030,1.364600,1.365025,1.364590,1.364800,2999.99,3002.57,2996.43,3001.83,299.39,299.68,299.01,299.58,5.25,5.26,5.23,5.25,11.19,11.30,11.17,11.28,33.09,33.13,33.07,33.12,54.77,54.78,54.71,54.77,13.75,13.94,13.62,13.66,22.21,22.23,22.21,22.23
29925,1.342075,1.344720,1.340560,1.344670,1.013080,1.013105,1.009675,1.010055,6.913620,6.914840,6.908465,6.911810,25.54020,25.54020,25.44495,25.46125,7.029040,7.029040,7.001540,7.005945,81.1,81.1,81.1,81.1,46.72,46.72,46.67,46.68,30.13,30.13,29.98,30.00,1.058375,1.062545,1.058375,1.061845,20.32,20.37,20.31,20.33,...,9.246470,9.246470,9.217525,9.229210,1.426835,1.426955,1.424485,1.424820,2199.20,2199.80,2194.51,2195.43,220.19,220.26,219.73,219.80,6.66,6.67,6.54,6.54,10.74,10.76,10.49,10.53,30.57,30.57,30.50,30.51,45.88,45.95,45.82,45.85,12.41,12.83,12.38,12.72,20.37,20.37,20.33,20.34
26790,1.314870,1.315155,1.314225,1.314265,0.989755,0.990270,0.989420,0.989845,6.562350,6.562885,6.560740,6.562830,24.10975,24.11835,24.09580,24.11145,6.631720,6.634185,6.627945,6.632085,77.8,77.8,77.8,77.8,46.43,46.48,46.41,46.47,25.90,25.92,25.82,25.82,1.121320,1.121960,1.120910,1.121255,19.32,19.33,19.29,19.31,...,8.347725,8.349105,8.342495,8.346850,1.378800,1.379585,1.378545,1.379440,2052.55,2053.17,2047.26,2048.33,205.52,205.61,204.98,205.15,5.29,5.31,5.28,5.29,11.65,11.67,11.64,11.66,28.18,28.19,28.14,28.14,48.57,48.58,48.46,48.48,15.70,15.87,15.69,15.82,18.86,18.86,18.83,18.83
48873,1.316365,1.316895,1.316150,1.316540,0.982325,0.983645,0.981895,0.982375,6.980390,6.980480,6.975315,6.978455,22.81740,22.83825,22.80830,22.81135,6.685740,6.691000,6.683155,6.684155,137.1,137.1,137.1,137.1,57.91,57.97,57.91,57.92,43.45,43.48,43.38,43.44,1.117735,1.118195,1.116885,1.118025,22.80,22.88,22.80,22.87,...,9.342565,9.354270,9.341505,9.351060,1.352465,1.352735,1.351645,1.352170,3163.72,3173.11,3163.14,3168.02,316.71,317.68,316.62,317.14,4.46,4.55,4.45,4.54,12.39,12.42,12.37,12.37,34.35,34.36,34.31,34.33,57.11,57.33,57.09,57.29,12.99,13.00,12.81,12.82,23.07,23.08,23.04,23.06
28338,1.291080,1.292430,1.290385,1.292045,0.963020,0.963520,0.962045,0.962205,6.662930,6.663745,6.659995,6.660765,23.90700,23.91420,23.89375,23.90645,6.583180,6.585300,6.579365,6.583295,78.9,78.9,78.9,78.9,49.88,49.88,49.82,49.82,28.48,28.50,28.44,28.46,1.130465,1.131105,1.130105,1.130440,20.82,20.85,20.81,20.83,...,8.381310,8.388740,8.379705,8.385190,1.351180,1.351730,1.350380,1.350795,2183.73,2185.00,2183.11,2183.66,218.61,218.75,218.53,218.57,5.06,5.07,5.02,5.05,11.34,11.34,11.30,11.33,29.70,29.70,29.67,29.68,48.93,49.01,48.92,48.96,12.81,12.83,12.61,12.66,19.85,19.85,19.85,19.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [12]:
pipe = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ("poly_features", PolynomialFeatures(degree=2, include_bias=False)),
    ("std_scaler", StandardScaler())
])

In [13]:
# export pipeline
pickle.dump(pipe, open("pipeline2.pkl", 'wb'))

In [14]:
prep_X_train = pipe.fit_transform(X_train)
prep_X_test = pipe.transform(X_test)
prep_X_val = pipe.transform(X_val)




In [15]:
model = Ridge(alpha=10,solver='cholesky')
model.fit(prep_X_train, y_train)
y_pred = model.predict(prep_X_test)
print("mean_squared_error: ",mean_squared_error(y_test, y_pred))


mean_squared_error:  8.379634832046879e-07


In [16]:
#Dump model
pickle.dump(model, open('model2.pkl', 'wb'))